In [1]:
import copy
import json
import math

import numpy as np

from fairseq.models.roberta import RobertaModel
import torch
import esm

from perplexity import compute_pl_logits, compute_scores, compute_pseudo_ppl, compute_boundaries

2022-03-15 16:36:29 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [114]:
# DO PADDING MASK!!
# Spatial breakdown of scores? Stratify by slices of length distribution

In [2]:
# modify this for data loading
data_path = "030922-joint-OVA-data-Golden-EpGrp-Jac.json"

with open(data_path, "r") as f:
    data = json.load(f)

examples = [(d["HC"], d["HC_AHo"]) for d in data]

In [4]:
model, alphabet = esm.pretrained.esm1_t6_43M_UR50S()

use_hc = True
for region in ["fr1", "cdr1", "fr2", "cdr2", "fr3", "cdr3", "fr4"]:
    scores = []
    for seq, aho in examples[:20]:
        start, end = compute_boundaries(aho, region, hc=use_hc)
        logits = compute_pl_logits(seq, alphabet, model, start, end)
        s_ = compute_scores(seq[start:end], alphabet, logits)
        scores.extend(s_)
    pppl = compute_pseudo_ppl(scores)
    print(f"{region}: {pppl}")

fr1: 6.993439968208059
cdr1: 4.851124093600071
fr2: 6.05599299529612
cdr2: 11.804280267366023
fr3: 5.069286801158001
cdr3: 21.466355094255473
fr4: 10.584183552105186


In [121]:
#model = RobertaModel.from_pretrained("biophiVH/", "checkpoint_best.pt") 

In [ ]:
model, alphabet = esm.pretrained.esm1_t6_43M_UR50S()

use_hc = True
for region in ["fr1", "cdr1", "fr2", "cdr2", "fr3", "cdr3", "fr4"]:
    scores = []
    for seq, aho in examples[:20]:
        start, end = compute_boundaries(aho, region, hc=use_hc)
        logits = compute_pl_logits(seq, alphabet, model, start, end)
        s_ = compute_scores(seq[start:end], logits)
        scores.extend(s_)
    pppl = compute_pseudo_ppl(scores)
    print(f"{region}: {pppl}")